In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

data = pd.read_csv('Datasets\\melb_data.csv')

#select target
y = data.Price

#using only numerical predictors
melbourne_predictors = data.drop(['Price'], axis=1)
X = melbourne_predictors.select_dtypes(exclude=['object'])

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

def score_dataset(X_train, X_val, y_train, y_val):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return mean_absolute_error(y_val, preds)

#score from Approach 1 (Drop columns with missing values)

#Get names of columns with missing values
cols_with_missing_vals = [col for col in X_train.columns
                          if X_train[col].isnull().any()]

#dropping columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing_vals, axis=1)
reduced_X_validation = X_val.drop(cols_with_missing_vals, axis=1)

print("MAE from Approach 1 (Drop columns with missing values): ")
print(score_dataset(reduced_X_train, reduced_X_validation, y_train, y_val))

#Score from Approach 2 (Imputation)

my_imputer = SimpleImputer()

imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_val = pd.DataFrame(my_imputer.fit_transform(X_val))

#Imputations removed column names so we add them back
imputed_X_train.columns = X_train.columns
imputed_X_val.columns = X_val.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_val, y_train, y_val))

#Making a copy to avoid changing orignal data when imputing
X_train_plus = X_train.copy()
X_val_plus = X_val.copy()

#New columns indicating what will be imputed
for col in cols_with_missing_vals:
    X_train_plus[col+'_was_missing'] = X_train_plus[col].isnull()
    X_val_plus[col+'_was_missing'] = X_val_plus[col].isnull()


my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_val_plus = pd.DataFrame(my_imputer.transform(X_val_plus))

imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_val_plus.columns = X_val_plus.columns

print("MAE from Approach 3 (An extension to Imputation): ")
print(score_dataset(imputed_X_train_plus, imputed_X_val_plus, y_train, y_val))


MAE from Approach 1 (Drop columns with missing values): 
183550.22137772635
MAE from Approach 2 (Imputation):
179816.89508731329
MAE from Approach 3 (An extension to Imputation): 


ValueError: could not convert string to float: 'Rooms'